In [23]:
import datetime
import json
import requests
import pandas as pd
from flask import Flask, request, redirect, g, render_template, session
from urllib.parse import quote

In [36]:
app = Flask(__name__)

#  Client Keys
CLIENT_ID = "fad471ed57bb445f9f5397411571544a"
CLIENT_SECRET = "b10027e90d9c4a6081fda3635808fe33"

# Spotify URLS
SPOTIFY_AUTH_URL = "https://accounts.spotify.com/authorize"
SPOTIFY_TOKEN_URL = "https://accounts.spotify.com/api/token"
SPOTIFY_API_BASE_URL = "https://api.spotify.com"
API_VERSION = "v1"
SPOTIFY_API_URL = "{}/{}".format(SPOTIFY_API_BASE_URL, API_VERSION)

# Server-side Parameters
CLIENT_SIDE_URL = "http://127.0.0.1"
PORT = 5050
REDIRECT_URI = "{}:{}/callback/q".format(CLIENT_SIDE_URL, PORT)
SCOPE = "playlist-modify-public playlist-modify-private playlist-read-private"
STATE = "plokiju123"
SHOW_DIALOG_bool = True
SHOW_DIALOG_str = str(SHOW_DIALOG_bool).lower()

auth_query_parameters = {
    "response_type": "code",
    "redirect_uri": REDIRECT_URI,
    "scope": SCOPE,
    "state": STATE,
    # "show_dialog": SHOW_DIALOG_str,
    "client_id": CLIENT_ID
}

@app.route("/")
def index():
    # Auth Step 1: Authorization
    url_args = "&".join(["{}={}".format(key, quote(val)) for key, val in auth_query_parameters.items()])
    auth_url = "{}/?{}".format(SPOTIFY_AUTH_URL, url_args)
    return redirect(auth_url)

@app.route("/callback/q")
def callback():
    # Auth Step 4: Requests refresh and access tokens
    auth_token = request.args['code']
    code_payload = {
        "grant_type": "authorization_code",
        "code": str(auth_token),
        "redirect_uri": REDIRECT_URI,
        'client_id': CLIENT_ID,
        'client_secret': CLIENT_SECRET,
    }
    post_request = requests.post(SPOTIFY_TOKEN_URL, data=code_payload)

    # Auth Step 5: Tokens are Returned to Application
    response_data = json.loads(post_request.text)
    access_token = response_data["access_token"]
    refresh_token = response_data["refresh_token"]
    token_type = response_data["token_type"]
    expires_in = response_data["expires_in"]

    # Auth Step 6: Use the access token to access Spotify API
    authorization_header = {"Authorization": "Bearer {}".format(access_token)}

    # Get profile data
    user_profile_api_endpoint = "{}/me".format(SPOTIFY_API_URL)
    profile_response = requests.get(user_profile_api_endpoint, headers=authorization_header)
    profile_data = json.loads(profile_response.text)

    # Get user playlist data
    playlist_api_endpoint = "{}/playlists".format(profile_data["href"])
    playlists_response = requests.get(playlist_api_endpoint, headers=authorization_header)
    playlist_data = json.loads(playlists_response.text)
    
    # Isolate official Discover Weekly playlist
    for item in playlist_data['items']:
        if((item['name'] == "Discover Weekly") & (item['owner']['display_name'] == "Spotify")):
            DISCOVER_WEEKLY_PLAYLIST_ID = item['id']
    
    # Get current Discover Weekly playlist tracks
    discover_weekly_playlist_tracks_endpoint = "{}/playlists/{}/tracks".format(SPOTIFY_API_URL, DISCOVER_WEEKLY_PLAYLIST_ID)
    discover_weekly_response = requests.get(discover_weekly_playlist_tracks_endpoint, headers=authorization_header)

    # Process current Discover Weekly tracks
    discover_weekly_track_details = discover_weekly_response.json()

    tracks = []
    sub_artists = []
    artists = []
    durations = []
    popularities = []
    release_dates = []
    track_ids = []

    for item in discover_weekly_track_details['items']:
        tracks.append(item['track']['name'])

    for item in discover_weekly_track_details['items']:
        track_ids.append(item['track']['id'])
        durations.append(datetime.datetime.fromtimestamp(item['track']['duration_ms']/1000).strftime('%-M:%S'))
        popularities.append(item['track']['popularity'])
        release_dates.append(item['track']['album']['release_date'])
        if(len(item['track']['artists'])>1):
            for artist in item['track']['artists']:
                sub_artists.append(artist['name'])
            artists.append(', '.join(sub_artists))
            sub_artists = []
        else:
            artists.append(item['track']['artists'][0]['name'])

    track_artist = list(zip(
        tracks,
        artists,
        durations,
        popularities,
        release_dates,
        track_ids,
    ))

    df = pd.DataFrame(track_artist, columns=[
        'track_name',
        'artists',
        'duration',
        'popularity',
        'release_date',
        'track_id',
    ])
    
    # Create Discover Archive playlist or identify it.
    
    
    discover_weekly_playlist_tracks_endpoint = "{}/playlists/{}/tracks".format(SPOTIFY_API_URL, DISCOVER_WEEKLY_PLAYLIST_ID)
    discover_weekly_response = requests.get(discover_weekly_playlist_tracks_endpoint, headers=authorization_header)
    
    create_playlist_endpoint = 'https://api.spotify.com/v1/users/{}/playlists'.format(profile_data['id'])
    
    request_body = json.dumps({
          "name": "Discover Archive test",
          "description": "Discover Archive test",
          "public": False
        })
    
    requests.post(url=create_playlist_endpoint, data=request_body, headers=authorization_header)

    return render_template('index.html', tables=[df.to_html(classes='data')], titles=df.columns.values)

In [37]:
if __name__ == "__main__":
    app.run(debug=False, port=PORT)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5050/ (Press CTRL+C to quit)
127.0.0.1 - - [01/Feb/2021 20:00:56] "GET / HTTP/1.1" 302 -
127.0.0.1 - - [01/Feb/2021 20:00:57] "GET /callback/q?code=AQDb0VOxDpmUHDLVfaJ5nJgVCJAa680LHHt-9G1YXf2KBx5TDr4Z1tfHhFPqWd--UKQU6Ocvi2qh926jO0ub0NocYfD1xMut26fDZhbl7wzti-BjV0T-F91Ebf3W043BWkNe_R2YEZgT79KaQlmzLlENAlVPqD2jvbJG4Y1o3jB05jAtbTbZot-tQagRuhVJMWO1h3yQqZ37PEaU3WY_uB_eEefWl2b3gs1ypZIFexn7HfeSrpcT-5-lK42edq_dmN0qk6czzjeX3fVf&state=plokiju123 HTTP/1.1" 200 -


In [38]:
import datetime
import json
import requests as re
import pandas as pd
from flask import Flask, request, redirect, g, render_template
from urllib.parse import quote

In [39]:
AUTH_URL = 'https://accounts.spotify.com/api/token'
AUTHORIZE_URL = 'https://accounts.spotify.com/authorize'
BASE_API_URL = 'https://api.spotify.com/v1/'
CLIENT_ID = 'fad471ed57bb445f9f5397411571544a'
CLIENT_SECRET = 'b10027e90d9c4a6081fda3635808fe33'
REDIRECT_URI = 'http://127.0.0.1:5000/'
MARKET = 'GB'
USERNAME = '12ian34'
SCOPES = 'playlist-modify-private playlist-read-private'

In [40]:
auth_response = re.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET,
})

auth_response_data = auth_response.json()
access_token = auth_response_data['access_token']
headers = {'Authorization': 'Bearer {token}'.format(token=access_token)}

In [41]:
get_user_profile = re.get(BASE_API_URL + 'users/' + USERNAME, headers=headers)
get_user_profile = get_user_profile.json()

In [45]:
# user_id = "YOUR_USER_ID"
# endpoint_url = f"https://api.spotify.com/v1/users/{user_id}/playlists"
# request_body = json.dumps({
#           "name": "Indie bands like Franz Ferdinand but using Python",
#           "description": "My first programmatic playlist, yooo!",
#           "public": False # let's keep it between us - for now
#         })
# response = requests.post(url = endpoint_url, data = request_body, headers={"Content-Type":"application/json", 
#                         "Authorization":"Bearer YOUR_TOKEN_HERE"})

In [46]:
get_user_playlists = re.get(BASE_API_URL + 'users/' + USERNAME + '/playlists/', headers=headers)
get_user_playlists = get_user_playlists.json()

for item in get_user_playlists['items']:
    if((item['name'] == "Discover Weekly") & (item['owner']['display_name'] == "Spotify")):
        DISCOVER_WEEKLY_PLAYLIST_ID = item['id']

In [72]:
get_user_playlists = re.get(BASE_API_URL + 'users/' + USERNAME + '/playlists/', headers=headers, params={'limit': 50})
get_user_playlists = get_user_playlists.json()

for item in get_user_playlists['items']:
    if((item['name'] == "Discover Archive test")):
        print(item)

In [73]:
get_user_playlists['next']

'https://api.spotify.com/v1/users/12ian34/playlists?offset=50&limit=50'

In [74]:
get_user_playlist_results = []

get_user_playlists = re.get(BASE_API_URL + 'users/' + USERNAME + '/playlists/', headers=headers, params={'limit': 50})
data = get_user_playlists.json()

# Store the first page of results
get_user_playlist_results += data

# While data['next'] isn't empty, let's download the next page, too
while data['next'] is not None:
    get_user_playlists = re.get(data['next'])
    data = get_user_playlists.json()
    # Store the current page of results
    get_user_playlist_results += data

KeyError: 'next'

In [64]:
get_user_playlists = re.get(BASE_API_URL + 'users/' + USERNAME + '/playlists/', headers=headers, params={'limit': 50})
get_user_playlists = get_user_playlists.json()

if(len(get_user_playlists['next']>0)):
    while len(get_user_playlists['next']>0):
        get_user_playlists = re.get(BASE_API_URL + 'users/' + USERNAME + '/playlists/', headers=headers, params={'limit': 50})


'https://api.spotify.com/v1/users/12ian34/playlists?offset=50&limit=50'

In [57]:
dir(get_user_playlists)

['__class__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'clear',
 'copy',
 'fromkeys',
 'get',
 'items',
 'keys',
 'pop',
 'popitem',
 'setdefault',
 'update',
 'values']

In [60]:
len(get_user_playlists['items'])

50

In [44]:
get_discover_weekly_tracks = re.get(BASE_API_URL + 'playlists/' + DISCOVER_WEEKLY_PLAYLIST_ID + '/tracks/?market=' + MARKET, headers=headers)
get_discover_weekly_tracks = get_discover_weekly_tracks.json()

tracks = []
sub_artists = []
artists = []
durations = []
popularities = []
release_dates = []
track_ids = []

for item in get_discover_weekly_tracks['items']:
    tracks.append(item['track']['name'])
    
for item in get_discover_weekly_tracks['items']:
    track_ids.append(item['track']['id'])
    durations.append(datetime.datetime.fromtimestamp(item['track']['duration_ms']/1000).strftime('%-M:%S'))
    popularities.append(item['track']['popularity'])
    release_dates.append(item['track']['album']['release_date'])
    if(len(item['track']['artists'])>1):
        for artist in item['track']['artists']:
            sub_artists.append(artist['name'])
        artists.append(', '.join(sub_artists))
        sub_artists = []
    else:
        artists.append(item['track']['artists'][0]['name'])

track_artist = list(zip(
    tracks,
    artists,
    durations,
    popularities,
    release_dates,
    track_ids,
))

df = pd.DataFrame(track_artist, columns=[
    'track_name',
    'artists',
    'duration',
    'popularity',
    'release_date',
    'track_id',
])

df

,track_name,artists,duration,popularity,release_date,track_id
0,Ordinary Boy - Floating Points Remix,"Ultraísta, Floating Points",7:33,28,2020-09-29,5BFn2BNYCQ42w3Z6P0MmIY
1,ʅ͡͡͡͡͡͡͡͡͡͡͡(ƟӨ)ʃ͡͡͡͡͡͡͡͡͡͡ ꐑ(ཀ ඊູ ఠీੂ೧ູ࿃ूੂ✧✧✧...,⣎⡇ꉺლ༽இ•̛)ྀ◞ ༎ຶ ༽ৣৢ؞ৢ؞ؖ ꉺლ,4:38,42,2019-10-31,2SJo1P387WJHJx1uFbyrRj
2,Throwing Down A Shape,Fila Brazillia,5:19,32,1998,0CFFq1tvJyVjtF9qc3pt5y
3,The C In Certain,David Grubbs,6:23,32,2018-04-01,1rNqvHHH5PTpX2E7guUwbj
4,Mohegan Suite,Daniel Lopatin,4:42,32,2019-12-13,5rB7OkSRElmSWoif0DNaIu
5,Seashell,Skylab,5:39,39,2015-06-09,18pedqpot9iF1Z7Ol3YHmc
6,Future Waves,Uther Moads,4:00,42,2011,1bL3lI3ykZjw9Zul1Ir51y
7,Abiogenesis,Carbon Based Lifeforms,6:45,39,2006-12-02,7B87dSMUgWM2Taj0EEXtjp
8,Nowhere,Nosaj Thing,2:28,24,2017-09-08,6IFxu0KyXQGvjvJ2bmaCZm
9,Fruitcake and Cookies,Sweet Trip,7:58,38,2003-06-10,50bVt9oMOYwnFe2uf4OSpf


In [ ]:
create_archive_playlist = re.get(BASE_API_URL + 'users/' + USERNAME + '/playlists/', headers=headers)


In [ ]:
https://api.spotify.com/v1/users/{user_id}/playlists

In [87]:
get_discover_weekly_tracks['items'][0]

{'added_at': '2021-02-01T00:00:00Z',
 'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/'},
  'href': 'https://api.spotify.com/v1/users/',
  'id': '',
  'type': 'user',
  'uri': 'spotify:user:'},
 'is_local': False,
 'primary_color': None,
 'track': {'album': {'album_type': 'single',
   'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/2f88S1uYsEwP0n4x36wvG7'},
     'href': 'https://api.spotify.com/v1/artists/2f88S1uYsEwP0n4x36wvG7',
     'id': '2f88S1uYsEwP0n4x36wvG7',
     'name': 'Ultraísta',
     'type': 'artist',
     'uri': 'spotify:artist:2f88S1uYsEwP0n4x36wvG7'}],
   'external_urls': {'spotify': 'https://open.spotify.com/album/3fobobpoaNdgD8XIMdqNMe'},
   'href': 'https://api.spotify.com/v1/albums/3fobobpoaNdgD8XIMdqNMe',
   'id': '3fobobpoaNdgD8XIMdqNMe',
   'images': [{'height': 640,
     'url': 'https://i.scdn.co/image/ab67616d0000b273fb79e88d6be5d46e5846e806',
     'width': 640},
    {'height': 300,
     'url': 'https://i.scdn

In [3]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id="39f9f9b5afca43ec900f2d7f7eb3bd67",
                                               client_secret="b4dafaa876a8407299f862250fecda35",
                                               redirect_uri="http://127.0.0.1:9090",
                                               scope="user-library-read"))

results = sp.current_user_saved_tracks()
for idx, item in enumerate(results['items']):
    track = item['track']
    print(idx, track['artists'][0]['name'], " – ", track['name'])

Couldn't read cache at: .cache


0 Fila Brazillia  –  Throwing Down A Shape
1 Underworld  –  Cups
2 The Durutti Column  –  Never Known
3 Call Super  –  Every Mouth Teeth Missing
4 36  –  Stasis Sounds for Long-Distance Space Travel (Stage 2)
5 Eliphino  –  I Don’t Care - Original Mix
6 Drake  –  Thank Me Now
7 Drake  –  Best I Ever Had
8 Drake  –  Up All Night
9 Drake  –  Over
10 Drake  –  Show Me A Good Time
11 deadmau5  –  Faxing Berlin
12 Vangelis  –  Blade Runner Blues
13 Sam Prekop  –  Summer Places
14 Holy Fuck  –  Shivering
15 Avey Tare  –  What’s The Goodside?
16 Overmono  –  Everything U Need
17 The Other People Place  –  Moonlight Rendezvous
18 HAAi  –  Rotating In Unison
19 NoGht  –  After Dark [long]


In [7]:
sp.user_playlist('12ian34')

{'collaborative': False,
 'description': None,
 'external_urls': {'spotify': 'https://open.spotify.com/user/12ian34/starred'},
 'followers': None,
 'href': None,
 'id': None,
 'images': [],
 'name': 'Starred',
 'owner': None,
 'primary_color': None,
 'public': True,
 'snapshot_id': None,
 'tracks': None,
 'type': 'playlist',
 'uri': None}